In [103]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import plot_roc_curve, confusion_matrix, ConfusionMatrixDisplay
from sklearn.datasets import load_wine
import matplotlib.pyplot as plt
import pandas as pd

In [104]:
# Carga de datos
train_input = np.asarray(pd.read_csv('train_data_input.csv', sep=',', header=None))

train_output = np.asarray(pd.read_csv('train_data_output.csv', sep=',', header=None))

test_input = np.asarray(pd.read_csv('test_data_input.csv', sep=',', header=None))

In [105]:
## Entrenamiento no supervisado con algoritmo K-means

# Agrupar puntos en clústers
k = 6 # Numero de neuronas en capa oculta
model = KMeans(k)
model.fit(train_input)
# Mostramos los puntos de los centroides (en un espacio R^8)
print("Centroides del modelo: \n" , model.cluster_centers_)

Centroides del modelo: 
 [[0.87395 0.5906  0.18955 0.87235 0.50445 0.44155 0.35895 0.51255]
 [0.16245 0.4587  0.38555 0.8604  0.72285 0.2547  0.2172  0.2844 ]
 [0.65645 0.33585 0.31695 0.44685 0.58395 0.24505 0.2895  0.27335]
 [0.5277  0.7041  0.5487  0.5432  0.8766  0.1965  0.2322  0.6819 ]
 [0.7902  0.6774  0.3495  0.3462  0.3223  0.1971  0.1831  0.7869 ]
 [0.1326  0.264   0.3441  0.1689  0.9041  0.3449  0.4359  0.1287 ]]


In [106]:
## Entrenamiento supervisado

# Extraer centroides
c = model.cluster_centers_

# Calcular el sigma
sigma = (max(c.flatten())-min(c.flatten()))/np.sqrt(2*k)

x = train_input
# Calcular matriz G
p = 9 # Cantidad de muestras

G = np.zeros((p,k))
for i in range(p):
    for j in range(k):
        dist = np.linalg.norm(x[i]-c[j], 2) # Distancia euclideana Entre Xi y Cj
        G[i,j] = np.exp((-1/(sigma**2))*dist**2) # Resultado de la función de activación para Gij

W = np.dot(np.linalg.pinv(G), train_output)

print("Pesos: \n" , W)

Pesos: 
 [[-4.09795507e-03  1.88830516e+00 -5.39040461e-04]
 [-1.33010485e-03  1.96196568e+00  1.95736887e+00]
 [ 2.84671302e+00 -3.54558899e-03 -6.76671304e-04]
 [-2.25289385e-04 -1.25731866e-03  1.00099883e+00]
 [-1.57121569e-04 -6.32321316e-05  9.99913306e-01]
 [ 9.99896906e-01 -1.85151049e-05  5.77311481e-06]]


In [107]:
## Predicción de la red

x = test_input
p = 3 # 3 datos de prueba
G = np.zeros((p,k))
for i in range(p):
    for j in range(k):
        dist = np.linalg.norm(x[i]-c[j], 2) # Distancia euclideana Entre Xi y Cj
        G[i,j] = np.exp((-1/(sigma**2))*dist**2) # Resultado de la función de activación para Gij

ynew = np.dot(G, W) # Salida de la red
print(ynew)

[[ 9.36816838e-01 -9.46518933e-04  2.01519759e-04]
 [ 3.16777951e-04  1.13638251e+00  7.75147238e-05]
 [ 8.21524487e-05  1.93941378e-05  9.90188751e-01]]


In [ ]:
## Cálculo del Error

test_output = np.asarray(pd.read_csv('test_data_output.csv', sep=',', header=None))
#error = (test_output - ynew)**2
error = 0
for i in range(3):
    dist = np.linalg.norm(test_output[i]-ynew[i], 2)
    errorprom = dist/3
print("Error: ")
print(errorprom)
precision = (1-errorprom)*100
print("Precisión: ")
print(precision)